In [1]:
import cv2 as cv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dlib
import tensorflow as tf

In [2]:
data = pd.read_csv('data/fer2013.csv')

In [11]:
# N = 1000
D = data

In [12]:
train_image = []
train_label = []
test_image = []
test_label = []

for index, row in D.iterrows():
    if row['Usage'] == 'Training':
        train_image.append(np.array(row['pixels'].split(), dtype=np.uint8))
        train_label.append(row['emotion'])
    else:
        test_image.append(np.array(row['pixels'].split(), dtype=np.uint8))
        test_label.append(row['emotion'])
        
train_image = np.array(train_image)
test_image = np.array(test_image)

train_label_one_hot = tf.Session().run(tf.one_hot(train_label, depth=7, on_value=1, off_value=0))
test_label_one_hot = tf.Session().run(tf.one_hot(test_label, depth=7, on_value=1, off_value=0))

In [13]:
import time

no_of_features = 48 * 48
no_of_classes = 7
x = tf.placeholder(tf.float32, [None, no_of_features])
W = tf.Variable(tf.zeros([no_of_features, no_of_classes]))
b = tf.Variable(tf.zeros([no_of_classes]))
y = tf.matmul(x, W) + b

# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, no_of_classes])

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

start_time = time.time()
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

print('tf init time - ', time.time() - start_time)

('tf init time - ', 0.0976400375366211)


In [14]:
start_time = time.time()
for _ in range(1000):
    if _ % 50 == 0:
        print(_)
    sess.run(train_step, feed_dict={x: train_image, y_: train_label_one_hot})

print('tf run time - ', time.time() - start_time)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
('tf run time - ', 622.0781359672546)


In [16]:
# Test trained model
start_time = time.time()
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: test_image, y_: test_label_one_hot}))

print('tf test time - ', time.time() - start_time)

0.255224
('tf test time - ', 0.25659990310668945)
